In [4]:
!pip install "dask[complete]"
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.5/205.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import the necessary Libraries

In [5]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import dask.dataframe as dd
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [6]:
pd.set_option('display.max_colwidth', None)

#Load and Preprocess Data

In [7]:
data_dir = Path('/content/drive/MyDrive/Data_hasking_without_banker_account')
ddf = dd.read_parquet(str(data_dir / '*.parquet'))
ddf

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,CONTRA_ACCNO,CONTRA_ZIP,CONTRA_NAME,CSHYN,REASON1,REASON2,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
npartitions=184,,,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,string,string,string,string,string,float64,float64,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,float64,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
df=ddf.compute()

In [9]:
# Optimize missing value handling
df = df.dropna(thresh=len(df) * 0.5, axis=1)  # Drop columns with more than 50% missing values
df = df.fillna(df.mode().iloc[0])  # Fill missing values with mode for string columns

# Identify numeric columns
numeric_cols = df.select_dtypes(include='number').columns

# Fill missing values with median for numeric columns only
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

In [8]:
df.isnull().sum()

CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO           0
ACC_CURRENCYISO          0
BUSINESSNO_TRANS         0
FK_CURRENCY              0
AMOUNT                   0
AMOUNTORIG               0
BRANCH_OFFICE            0
CONTRA_ACCNO             0
CONTRA_ZIP               0
CONTRA_NAME              0
CSHYN                    0
REASON1                  0
REASON2                  0
TR_FLAG_01               0
ANALYTICAL_TRANS_CODE    0
TR_SP_01                 0
TR_SP_02                 0
TR_SP_03                 0
TR_SP_04                 0
TR_SP_05                 0
VALUEDATE                0
dtype: int64

#Label encoding

In [10]:
# Label encode string columns
label_encoders = {}
for column in df.select_dtypes(include=['string']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [11]:
df

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON1,REASON2,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,VALUEDATE
709,307853,0,124557,0,12,57951,12,414.5,414.5,0,...,712708,801689,0,5,4274211,11,63,2,15985,0
894,353656,0,160657,0,12,12797,12,-510.0,-510.0,0,...,424297,674913,0,1,3952059,7,63,0,188373,0
902,76980,0,385399,0,12,44289,12,500.0,500.0,0,...,594131,229221,0,2,3174201,7,63,0,220605,0
904,353656,0,160657,0,12,30638,12,1000.0,1000.0,0,...,385402,518268,0,2,3865242,7,63,0,153984,0
907,76980,0,385399,0,12,5843,12,-500.0,-500.0,0,...,163310,621458,0,1,3639232,7,63,0,220605,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031051,423673,0,10543,0,12,11269503,12,-5700.0,-5700.0,0,...,356078,518514,0,1,814652,7,63,0,168775,179
1031052,62869,0,10543,0,12,11321626,12,2500.0,2500.0,0,...,310915,180154,0,2,2008719,7,63,0,119744,179
1031056,395105,0,10543,0,12,11270712,12,-1000.0,-1000.0,0,...,553774,782221,0,1,4519756,7,63,0,206472,179
1031060,400980,0,10543,0,12,11324157,12,13387.1,13387.1,0,...,78003,452681,1,2,2885479,8,63,0,178409,179


#Apply the FP_Growth

In [29]:

frequent_itemsets = fpgrowth(df, min_support=0.99, use_colnames=True)

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 307853.0

**---> the algorithm  expects a DataFrame containing only boolean or binary integer values (0 or 1).**

In [11]:
# Create a binary representation of the data
frequent_itemsets = fpgrowth(df.applymap(lambda x: 1 if x else 0), min_support=0.1, use_colnames=True)

In [13]:
frequent_itemsets

,support,itemsets
0,1.000000,(BUSINESSNO_TRANS)
1,1.000000,(REASON1)
2,1.000000,(CONTRA_NAME)
3,1.000000,(TR_SP_05)
4,1.000000,(REASON2)
...,...,...
262138,0.993089,"(ACC_ACCNO, CUST_CUSTNO, BUSINESSNO_TRANS, REASON2, VALUEDATE, CONTRA_NAME, ANALYTICAL_TRANS_CODE, TR_SP_02, TR_SP_03, FK_CURRENCY, CONTRA_ACCNO, TR_SP_01, TR_SP_05, ACC_CURRENCYISO)"
262139,0.993089,"(ACC_ACCNO, CUST_CUSTNO, BUSINESSNO_TRANS, REASON2, REASON1, VALUEDATE, CONTRA_NAME, ANALYTICAL_TRANS_CODE, TR_SP_02, TR_SP_03, FK_CURRENCY, CONTRA_ACCNO, TR_SP_01, ACC_CURRENCYISO)"
262140,0.993089,"(ACC_ACCNO, CUST_CUSTNO, BUSINESSNO_TRANS, REASON2, REASON1, VALUEDATE, CONTRA_NAME, ANALYTICAL_TRANS_CODE, TR_SP_02, TR_SP_03, CONTRA_ACCNO, TR_SP_01, TR_SP_05, ACC_CURRENCYISO)"
262141,0.993089,"(ACC_ACCNO, CUST_CUSTNO, REASON2, REASON1, VALUEDATE, CONTRA_NAME, ANALYTICAL_TRANS_CODE, TR_SP_02, TR_SP_03, FK_CURRENCY, CONTRA_ACCNO, TR_SP_01, TR_SP_05, ACC_CURRENCYISO)"


In [12]:
# Create a binary representation of the data
frequent_itemsets1 = fpgrowth(df.applymap(lambda x: 1 if x else 0), min_support=0.9, use_colnames=True)

In [13]:
frequent_itemsets1

,support,itemsets
0,1.000000,(REASON1)
1,1.000000,(BUSINESSNO_TRANS)
2,1.000000,(CONTRA_NAME)
3,1.000000,(REASON2)
4,1.000000,(TR_SP_05)
...,...,...
131066,0.993089,"(TR_SP_01, ACC_CURRENCYISO, FK_CURRENCY, TR_SP_03, TR_SP_05, ACC_ACCNO, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, BUSINESSNO_TRANS, TR_SP_02, ANALYTICAL_TRANS_CODE)"
131067,0.993089,"(TR_SP_01, ACC_CURRENCYISO, FK_CURRENCY, TR_SP_03, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, BUSINESSNO_TRANS, TR_SP_02, ANALYTICAL_TRANS_CODE)"
131068,0.993089,"(TR_SP_01, FK_CURRENCY, TR_SP_03, TR_SP_05, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, BUSINESSNO_TRANS, TR_SP_02, ANALYTICAL_TRANS_CODE)"
131069,0.993089,"(TR_SP_01, ACC_CURRENCYISO, FK_CURRENCY, TR_SP_03, TR_SP_05, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)"


# Generate association rules

In [31]:
# Generate association rules from the frequent itemsets
rules1 = association_rules(frequent_itemsets1, metric="confidence", min_threshold=1)

In [39]:
# prompt: calculate the min_spport and confidence for this data

# Calculate the minimum support for the frequent itemsets
min_support = frequent_itemsets1['support'].min()

# Calculate the minimum confidence for the association rules
min_confidence = rules1['confidence'].min()

print(f"Minimum support: {min_support}")
print(f"Minimum confidence: {min_confidence}")


Minimum support: 0.9845125069586946
Minimum confidence: 1.0


In [33]:
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(TR_SP_05),(CONTRA_NAME),1.000000,1.000000,1.000000,1.0,1.000000,1.763369e-07,inf,1.000000
1,(CONTRA_NAME),(TR_SP_05),1.000000,1.000000,1.000000,1.0,1.000000,1.763369e-07,inf,1.000000
2,"(TR_SP_05, BUSINESSNO_TRANS)",(CONTRA_NAME),1.000000,1.000000,1.000000,1.0,1.000000,1.763368e-07,inf,0.666667
3,"(CONTRA_NAME, BUSINESSNO_TRANS)",(TR_SP_05),1.000000,1.000000,1.000000,1.0,1.000000,1.763368e-07,inf,0.666667
4,"(REASON1, TR_SP_05)",(CONTRA_NAME),1.000000,1.000000,1.000000,1.0,1.000000,1.763368e-07,inf,0.666667
...,...,...,...,...,...,...,...,...,...,...
477179,"(TR_SP_01, TR_SP_03, FK_CURRENCY, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, BUSINESSNO_TRANS, TR_SP_02, ANALYTICAL_TRANS_CODE)","(ACC_CURRENCYISO, TR_SP_05)",0.993089,0.999863,0.993089,1.0,1.000137,1.362420e-04,inf,0.019851
477180,"(TR_SP_01, TR_SP_03, ACC_CURRENCYISO, TR_SP_05, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)","(BUSINESSNO_TRANS, FK_CURRENCY, CONTRA_NAME)",0.993089,0.999863,0.993089,1.0,1.000137,1.363295e-04,inf,0.019864
477181,"(TR_SP_01, TR_SP_03, ACC_CURRENCYISO, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)","(FK_CURRENCY, TR_SP_05, BUSINESSNO_TRANS)",0.993089,0.999863,0.993089,1.0,1.000137,1.363295e-04,inf,0.019864
477182,"(TR_SP_01, TR_SP_03, FK_CURRENCY, TR_SP_05, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, CONTRA_NAME)",0.993089,0.999863,0.993089,1.0,1.000137,1.363295e-04,inf,0.019864


In [46]:
rules1.sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
423581,"(TR_SP_01, TR_SP_03, FK_CURRENCY, TR_SP_05, REASON1, CONTRA_ACCNO, AMOUNT, VALUEDATE, TR_SP_02)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, CONTRA_NAME, AMOUNTORIG)",0.984519,0.991279,0.984519,1.0,1.008797,8.585653e-03,inf,0.563330
423582,"(TR_SP_01, TR_SP_03, FK_CURRENCY, REASON1, CONTRA_ACCNO, AMOUNTORIG, CONTRA_NAME, VALUEDATE, TR_SP_02)","(AMOUNT, ACC_CURRENCYISO, TR_SP_05, BUSINESSNO_TRANS)",0.984519,0.991279,0.984519,1.0,1.008797,8.585653e-03,inf,0.563330
423579,"(TR_SP_01, TR_SP_03, ACC_CURRENCYISO, REASON1, CONTRA_ACCNO, CONTRA_NAME, AMOUNT, VALUEDATE, TR_SP_02)","(BUSINESSNO_TRANS, FK_CURRENCY, TR_SP_05, AMOUNTORIG)",0.984519,0.991279,0.984519,1.0,1.008797,8.585653e-03,inf,0.563330
361260,"(FK_CURRENCY, TR_SP_05, CUST_CUSTNO, AMOUNTORIG, VALUEDATE, TR_SP_02)","(BUSINESSNO_TRANS, AMOUNT, ACC_CURRENCYISO, CONTRA_NAME)",0.984928,0.991279,0.984928,1.0,1.008797,8.589214e-03,inf,0.578595
378351,"(TR_SP_01, FK_CURRENCY, ACC_ACCNO, CONTRA_NAME, AMOUNT, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, TR_SP_05, AMOUNTORIG)",0.984923,0.991279,0.984923,1.0,1.008797,8.589174e-03,inf,0.578419
...,...,...,...,...,...,...,...,...,...,...
15615,"(ACC_CURRENCYISO, TR_SP_05, FK_CURRENCY, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, TR_SP_02)",(BUSINESSNO_TRANS),0.998879,1.000000,0.998879,1.0,1.000000,8.806959e-08,inf,0.000079
427962,"(TR_SP_01, TR_SP_03, ACC_CURRENCYISO, FK_CURRENCY, REASON1, CONTRA_ACCNO, AMOUNTORIG, CONTRA_NAME, AMOUNT, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)",(BUSINESSNO_TRANS),0.984518,1.000000,0.984518,1.0,1.000000,8.680341e-08,inf,0.000006
15554,"(FK_CURRENCY, TR_SP_05, CUST_CUSTNO, CONTRA_ACCNO, TR_SP_02)",(BUSINESSNO_TRANS),0.998879,1.000000,0.998879,1.0,1.000000,8.806960e-08,inf,0.000079
15557,"(FK_CURRENCY, TR_SP_05, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, TR_SP_02)",(BUSINESSNO_TRANS),0.998879,1.000000,0.998879,1.0,1.000000,8.806960e-08,inf,0.000079


In [38]:
# prompt: filter the rules by the best values of zhangs_metric and lift

best_rules = rules1[(rules1['zhangs_metric'] > 1) & (rules1['lift'] > 1)]


In [39]:
best_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(TR_SP_05),(CONTRA_NAME),1.000000,1.000000,1.000000,1.0,1.000000,1.763369e-07,inf,1.0
1,(CONTRA_NAME),(TR_SP_05),1.000000,1.000000,1.000000,1.0,1.000000,1.763369e-07,inf,1.0
1096,"(TR_SP_05, FK_CURRENCY)","(ACC_CURRENCYISO, CONTRA_NAME)",0.999863,0.999863,0.999863,1.0,1.000137,1.371713e-04,inf,1.0
1097,"(FK_CURRENCY, CONTRA_NAME)","(TR_SP_05, ACC_CURRENCYISO)",0.999863,0.999863,0.999863,1.0,1.000137,1.371713e-04,inf,1.0
1098,"(TR_SP_05, ACC_CURRENCYISO)","(FK_CURRENCY, CONTRA_NAME)",0.999863,0.999863,0.999863,1.0,1.000137,1.371713e-04,inf,1.0
1099,"(ACC_CURRENCYISO, CONTRA_NAME)","(TR_SP_05, FK_CURRENCY)",0.999863,0.999863,0.999863,1.0,1.000137,1.371713e-04,inf,1.0
98304,(AMOUNT),(AMOUNTORIG),0.991417,0.991417,0.991417,1.0,1.008658,8.509700e-03,inf,1.0
98305,(AMOUNTORIG),(AMOUNT),0.991417,0.991417,0.991417,1.0,1.008658,8.509700e-03,inf,1.0


**Filter rules by 'CONTRA_ACCNO' or 'CUST_CUSTNO'**





In [67]:
# Focus on rules involving 'CONTRA_ACCNO' or 'CUST_CUSTNO'

rules_with_contra_accno_or_cust_custno = rules1[
    (rules1['antecedents'].apply(lambda x: 'CONTRA_ACCNO' in x or 'CUST_CUSTNO' in x)) |
    (rules1['consequents'].apply(lambda x: 'CONTRA_ACCNO' in x or 'CUST_CUSTNO' in x))
]

rules_with_contra_accno_or_cust_custno


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
32,"(CUST_CUSTNO, TR_SP_05)",(CONTRA_NAME),0.999998,1.000000,0.999998,1.0,1.000000,1.763366e-07,inf,0.111111
33,"(CUST_CUSTNO, CONTRA_NAME)",(TR_SP_05),0.999998,1.000000,0.999998,1.0,1.000000,1.763366e-07,inf,0.111111
34,"(REASON1, CUST_CUSTNO, TR_SP_05)",(CONTRA_NAME),0.999998,1.000000,0.999998,1.0,1.000000,1.763366e-07,inf,0.105263
35,"(REASON1, CUST_CUSTNO, CONTRA_NAME)",(TR_SP_05),0.999998,1.000000,0.999998,1.0,1.000000,1.763366e-07,inf,0.105263
36,"(CUST_CUSTNO, TR_SP_05, BUSINESSNO_TRANS)",(CONTRA_NAME),0.999998,1.000000,0.999998,1.0,1.000000,1.763366e-07,inf,0.105263
...,...,...,...,...,...,...,...,...,...,...
477179,"(TR_SP_01, TR_SP_03, FK_CURRENCY, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, BUSINESSNO_TRANS, TR_SP_02, ANALYTICAL_TRANS_CODE)","(ACC_CURRENCYISO, TR_SP_05)",0.993089,0.999863,0.993089,1.0,1.000137,1.362420e-04,inf,0.019851
477180,"(TR_SP_01, TR_SP_03, ACC_CURRENCYISO, TR_SP_05, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)","(BUSINESSNO_TRANS, FK_CURRENCY, CONTRA_NAME)",0.993089,0.999863,0.993089,1.0,1.000137,1.363295e-04,inf,0.019864
477181,"(TR_SP_01, TR_SP_03, ACC_CURRENCYISO, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, CONTRA_NAME, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)","(FK_CURRENCY, TR_SP_05, BUSINESSNO_TRANS)",0.993089,0.999863,0.993089,1.0,1.000137,1.363295e-04,inf,0.019864
477182,"(TR_SP_01, TR_SP_03, FK_CURRENCY, TR_SP_05, ACC_ACCNO, REASON1, CUST_CUSTNO, CONTRA_ACCNO, REASON2, VALUEDATE, TR_SP_02, ANALYTICAL_TRANS_CODE)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, CONTRA_NAME)",0.993089,0.999863,0.993089,1.0,1.000137,1.363295e-04,inf,0.019864


In [77]:
# Sorting by lift and Zhang's metric in descending order
sorted_rules = rules_with_contra_accno_or_cust_custno.sort_values(by=['lift', 'zhangs_metric'], ascending=[False, False])

# Selecting the top 10 rules
top_10_rules = sorted_rules.head(10)

# Display the top 10 rules
top_10_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
174568,"(ACC_CURRENCYISO, TR_SP_05, CUST_CUSTNO, AMOUNTORIG, TR_SP_02)","(BUSINESSNO_TRANS, AMOUNT, FK_CURRENCY, CONTRA_NAME)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174569,"(ACC_CURRENCYISO, TR_SP_05, CUST_CUSTNO, AMOUNT, TR_SP_02)","(BUSINESSNO_TRANS, FK_CURRENCY, CONTRA_NAME, AMOUNTORIG)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174570,"(ACC_CURRENCYISO, CUST_CUSTNO, CONTRA_NAME, AMOUNTORIG, TR_SP_02)","(AMOUNT, FK_CURRENCY, TR_SP_05, BUSINESSNO_TRANS)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174571,"(ACC_CURRENCYISO, CUST_CUSTNO, CONTRA_NAME, AMOUNT, TR_SP_02)","(BUSINESSNO_TRANS, FK_CURRENCY, TR_SP_05, AMOUNTORIG)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174572,"(FK_CURRENCY, TR_SP_05, CUST_CUSTNO, AMOUNTORIG, TR_SP_02)","(BUSINESSNO_TRANS, AMOUNT, ACC_CURRENCYISO, CONTRA_NAME)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174573,"(FK_CURRENCY, TR_SP_05, CUST_CUSTNO, AMOUNT, TR_SP_02)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, CONTRA_NAME, AMOUNTORIG)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174574,"(FK_CURRENCY, CUST_CUSTNO, CONTRA_NAME, AMOUNTORIG, TR_SP_02)","(AMOUNT, ACC_CURRENCYISO, TR_SP_05, BUSINESSNO_TRANS)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174575,"(FK_CURRENCY, CUST_CUSTNO, CONTRA_NAME, AMOUNT, TR_SP_02)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, TR_SP_05, AMOUNTORIG)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899058
174744,"(ACC_CURRENCYISO, TR_SP_05, REASON1, CUST_CUSTNO, AMOUNTORIG, TR_SP_02)","(BUSINESSNO_TRANS, AMOUNT, FK_CURRENCY, CONTRA_NAME)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899050
174745,"(ACC_CURRENCYISO, TR_SP_05, REASON1, CUST_CUSTNO, AMOUNT, TR_SP_02)","(BUSINESSNO_TRANS, FK_CURRENCY, CONTRA_NAME, AMOUNTORIG)",0.9903,0.991279,0.9903,1.0,1.008797,0.008636,inf,0.899050
